In [1]:
import torch
from rich.markdown import Markdown

device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
%cd ..

/home/oscarn/flan-gpt2


In [3]:
from models.model_utils import T5Model
# model_name = "OscarNav/flan-flan-t5-large-ft"
model_name = "google/flan-t5-xl"

wrapped_model = T5Model(model_name, device, quantization=True)
model = wrapped_model.get_model()
# model.resize_token_embeddings(len(wrapped_model.get_tokenizer()))
tokenizer = wrapped_model.get_tokenizer()

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2025-09-07 23:23:36,076 - accelerate.big_modeling - WARNING - You shouldn't move a model that is dispatched using accelerate hooks.


In [5]:
if torch.cuda.is_available():
    print(f"{torch.cuda.memory_allocated() / 1e9} GB allocated")

3.511022592 GB allocated


In [7]:
from eval.eval_utils import Evaluation
from data.data_utils import create_instruct_dataset


eval = Evaluation(model, tokenizer, device)

In [8]:
num_samples = 100
CAUSAL_LM = False

In [9]:
template = """
## INPUT PROMPT
{prompt}
## GROUND_TRUTH
{completion}
## MODEL OUTPUT
{prediction}
"""

### ANLI

In [10]:
dataset = create_instruct_dataset(num_samples, ["anli"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Map:   0%|          | 0/16946 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16946 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  2.36it/s]


INPUT PROMPT                                                    

Lee Il-hyung is a South Korean film director and screenwriter. Prior to directing his first feature film "A Violent
Prosecutor" (2016), Lee is an assistant director on films, such as "The Moonlight of Seoul" (2008), "My Way" (2011)
and "" (2014), and commercial films.                                                                               

Based on the paragraph above can we conclude that "Il-Hung was born in China"?                                     

OPTIONS:                                                                                                           

 • entailment                                                                                                      
 • neutral                                                                                                         
 • contradiction                                                                                                   


                                                   GROUND_TRUTH                                                    

contradiction                                                                                                      


                                                   MODEL OUTPUT                                                    

neutral

## Common Gen

In [12]:
dataset = create_instruct_dataset(num_samples, ["common_gen"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Generating responses... : 100%|██████████| 1/1 [00:01<00:00,  1.40s/it]


INPUT PROMPT                                                    

What are the most important words in the following sentence:                                                       

A skier making her way down a snowy mountain.                                                                      


                                                   GROUND_TRUTH                                                    

['mountain', 'skier', 'way']                                                                                       


                                                   MODEL OUTPUT                                                    

mountain, skier, way

## SQUAD

In [12]:
dataset = create_instruct_dataset(num_samples, ["squad"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Using the latest cached version of the dataset since rajpurkar/squad couldn't be found on the Hugging Face Hub
2025-09-07 22:38:59,979 - datasets.load - WARNING - Using the latest cached version of the dataset since rajpurkar/squad couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'plain_text' at /home/oscarn/.cache/huggingface/datasets/rajpurkar___squad/plain_text/0.0.0/7b6d24c440a36b6815f21b70d25016731768db1f (last modified on Fri Nov 22 00:36:36 2024).
2025-09-07 22:38:59,985 - datasets.packaged_modules.cache.cache - WARNING - Found the latest cached dataset configuration 'plain_text' at /home/oscarn/.cache/huggingface/datasets/rajpurkar___squad/plain_text/0.0.0/7b6d24c440a36b6815f21b70d25016731768db1f (last modified on Fri Nov 22 00:36:36 2024).


Filter:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/87241 [00:00<?, ? examples/s]

Filter:   0%|          | 0/87241 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  1.65it/s]


INPUT PROMPT                                                    

In 2015-2016, Notre Dame ranked 18th overall among "national universities" in the United States in U.S. News &     
World Report's Best Colleges 2016. In 2014, USA Today ranked Notre Dame 10th overall for American universities     
based on data from College Factual. Forbes.com's America's Best Colleges ranks Notre Dame 13th among colleges in   
the United States in 2015, 8th among Research Universities, and 1st in the Midwest. U.S. News & World Report also  
lists Notre Dame Law School as 22nd overall. BusinessWeek ranks Mendoza College of Business undergraduate school as
1st overall. It ranks the MBA program as 20th overall. The Philosophical Gourmet Report ranks Notre Dame's graduate
philosophy program as 15th nationally, while ARCHITECT Magazine ranked the undergraduate architecture program as   
12th nationally. Additionally, the study abroad program ranks sixth in highest participation percentage in the     
nation, with 57.6% of students choosing to study abroad in 17 countries. According to payscale.com, undergraduate  
alumni of University of Notre Dame have a mid-career median salary $110,000, making it the 24th highest among      
colleges and universities in the United States. The median starting salary of $55,300 ranked 58th in the same peer 
group. In 2014 what entity named Notre Dame 10th best of all American universities?                                


                                                   GROUND_TRUTH                                                    

USA Today                                                                                                          


                                                   MODEL OUTPUT                                                    

USA Today

## Cosmos QA

In [13]:
dataset = create_instruct_dataset(num_samples, ["cosmos_qa"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Using the latest cached version of the module from /home/oscarn/.cache/huggingface/modules/datasets_modules/datasets/allenai--cosmos_qa/3e18538cbfdb2c04189b16642715f0f6da3e97ed5df0aadcec3641245b2cf157 (last modified on Fri Nov 22 00:37:03 2024) since it couldn't be found locally at allenai/cosmos_qa, or remotely on the Hugging Face Hub.
2025-09-07 22:40:36,715 - datasets.load - WARNING - Using the latest cached version of the module from /home/oscarn/.cache/huggingface/modules/datasets_modules/datasets/allenai--cosmos_qa/3e18538cbfdb2c04189b16642715f0f6da3e97ed5df0aadcec3641245b2cf157 (last modified on Fri Nov 22 00:37:03 2024) since it couldn't be found locally at allenai/cosmos_qa, or remotely on the Hugging Face Hub.


Map:   0%|          | 0/25262 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25262 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:01<00:00,  1.27s/it]


INPUT PROMPT                                                    

Read the following article and answer the question.                                                                

I also got a really cute shirt from a classmate . Today I will get party light candles . The women who is giving   
them to me wanted to know what sent I wanted . I am getting Wild Fern .                                            

Why did I get a cute t - shirt from a classmate and a wild fern scented candle from a woman ? OPTIONS:             

 • It is my birthday and I got unpleasant gifts from friends .                                                     
 • None of the above choices .                                                                                     
 • It is Christmas and I gave unpleasant gifts to friends .                                                        
 • It is my birthday and I gave nice gifts to friends .                                                            


                                                   GROUND_TRUTH                                                    

None of the above choices .                                                                                        


                                                   MODEL OUTPUT                                                    

None of the above choices.

## CoQA

In [14]:
dataset = create_instruct_dataset(num_samples, ["coqa"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], max_tokens=150, return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/7199 [00:00<?, ? examples/s]

Map:   0%|          | 0/4063 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4063 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:12<00:00, 12.47s/it]


INPUT PROMPT                                                    

CHAPTER XXXVI.                                                                                                     

TOM TRINGLE GETS AN ANSWER.                                                                                        

Faddle as he went down into the country made up his mind that the law which required such letters to be delivered  
by hand was an absurd law. The post would have done just as well, and would have saved a great deal of trouble.    
These gloomy thoughts were occasioned by a conviction that he could not carry himself easily or make himself happy 
among such "howling swells" as these Alburys. If they should invite him to the house the matter would be worse that
way than the other. He had no confidence in his dress coat, which he was aware had been damaged by nocturnal       
orgies. It is all very well to tell a fellow to be as "big a swell" as anybody else, as Tom had told him. But      
Faddle acknowledged to himself the difficulty of acting up to such advice. Even the eyes of Colonel Stubbs turned  
upon him after receipt of the letter would oppress him.                                                            

Nevertheless he must do his best, and he took a gig at the station nearest to Albury. He was careful to carry his  
bag with him, but still he lived in hope that he would be able to return to London the same day. When he found     
himself within the lodges of Stalham Park he could hardly keep himself from shivering, and, when he asked the      
footman at the door whether Colonel Stubbs were there, he longed to be told that Colonel Stubbs had gone away on   
the previous day to some--he did not care what--distant part of the globe. But Colonel Stubbs had not gone away.   
Colonel Stubbs was in the house.                                                                                   

  1 What is this chapter called?                                                                                   
  2 Who took a job at a station by Albury?                                                                         
  3 Did he hope to stay there?                                                                                     
  4 What did he hope?                                                                                              
  5 What did he take along?                                                                                        
  6 Who did he ask about?                                                                                          
  7 What was his question?                                                                                         
  8 Who did he ask?                                                                                                
  9 What was the answer?                                                                                           
 10 What was he hoping?                                                                                            
 11 What did Faddle think was absurd?                                                                              
 12 Did he think the mail would be just as good?                                                                   
 13 What would be a worse matter?                                                                                  


                                                   GROUND_TRUTH                                                    

  1 TOM TRINGLE GETS AN ANSWER.                                                                                    
  2 Faddle.                                                                                                        
  3 No.                                                                                                            
  4 That he would be able to return to London.                                                     

## XSum

In [15]:
dataset = create_instruct_dataset(num_samples, ["xsum"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Using the latest cached version of the dataset since EdinburghNLP/xsum couldn't be found on the Hugging Face Hub
2025-09-07 22:41:22,517 - datasets.load - WARNING - Using the latest cached version of the dataset since EdinburghNLP/xsum couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/oscarn/.cache/huggingface/datasets/EdinburghNLP___xsum/default/1.2.0/40db7604fedb616a9d2b0673d11838fa5be8451c (last modified on Sun Sep  7 16:46:46 2025).
2025-09-07 22:41:22,524 - datasets.packaged_modules.cache.cache - WARNING - Found the latest cached dataset configuration 'default' at /home/oscarn/.cache/huggingface/datasets/EdinburghNLP___xsum/default/1.2.0/40db7604fedb616a9d2b0673d11838fa5be8451c (last modified on Sun Sep  7 16:46:46 2025).


Filter:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:04<00:00,  4.04s/it]


INPUT PROMPT                                                    

Article: Varela, 20, had a trial at Old Trafford last season and is David Moyes's first signing as Manchester      
United manager. The defender has made one appearance for Penarol but has nine caps for Uruguay's Under-20 side.    
"I'm very pleased to be a part of this club, one of the best in the world," he said after signing a five-year deal.
"As everyone in the world knows, this is a huge club that has won everything and I really hope that continues."    
Varela is taking part in the Under-20 World Championships, which runs from 21 June to 13 July, in Turkey. Last     
week, Penarol head coach Jorge Da Silva, who is reported to have since resigned, said he believed the youngster has
earned the move. Da Silva said: "This is what he deserves. It is a shame to see him go but you can't deny him the  
opportunity to join a club like this." Rafael was United's regular right-back last season, with Phil Jones and     
Chris Smalling also featuring in the role.                                                                         

A summary of the above article is?                                                                                 


                                                   GROUND_TRUTH                                                    

Manchester United have completed the signing Uruguayan right-back Guillermo Varela from Atletico Penarol for an    
undisclosed fee.                                                                                                   


                                                   MODEL OUTPUT                                                    

Uruguayan right-back Dani Varela has signed a five-year contract to join Manchester United from rival club Penarol.

## BoolQ

In [16]:
dataset = create_instruct_dataset(num_samples, ["bool_q"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/9427 [00:00<?, ? examples/s]

Map:   0%|          | 0/9399 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9399 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  1.33it/s]


INPUT PROMPT                                                    

In terms of the parts of the grain (the grass fruit) used in flour--the endosperm or protein/starchy part, the germ
or protein/fat/vitamin-rich part, and the bran or fiber part--there are three general types of flour. White flour  
is made from the endosperm only. Brown flour includes some of the grain's germ and bran, while whole grain or      
wholemeal flour is made from the entire grain, including the bran, endosperm, and germ. Germ flour is made from the
endosperm and germ, excluding the bran.                                                                            

Is the following statement correct based on the text                                                               

is wheat flour and white flour the same thing                                                                      

OPTIONS:                                                                                                           

 • True                                                                                                            
 • False                                                                                                           


                                                   GROUND_TRUTH                                                    

False                                                                                                              


                                                   MODEL OUTPUT                                                    

False

## Python Code

In [17]:
dataset = create_instruct_dataset(num_samples, ["python_code"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], max_tokens=1000, return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/16750 [00:00<?, ? examples/s]

Map:   0%|          | 0/16237 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16237 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:49<00:00, 49.67s/it]


INPUT PROMPT                                                    

Create a Python neural network model that takes in a given trajectory of stock prices and predicts the stock price 
for the next day. [10, 10.4, 10.5, 10.6, 10.7, 10.9, 11]                                                           


                                                   GROUND_TRUTH                                                    

                                                                                                                   
 import numpy as np                                                                                                
 import tensorflow as tf                                                                                           
                                                                                                                   
 def create_model():                                                                                               
     model = tf.keras.Sequential([                                                                                 
         tf.keras.layers.Dense(64, activation='relu', input_shape=(7, )),                                          
         tf.keras.layers.Dense(1)                                                                                  
     ])                                                                                                            
                                                                                                                   
     model.compile(loss='mean_squared_error',                                                                      
                   optimizer='adam',                                                                               
                   metrics=['accuracy'])                                                                           
                                                                                                                   
     return model                                                                                                  
                                                                                                                   
 if __name__ == "__main__":                                                                                        
     model = create_model()                                                                                        
                                                                                                                   
     input_data = np.array([[10, 10.4, 10.5, 10.6, 10.7, 10.9, 11]])                                               
     predictions = model.predict(input_data)                                                                       
                                                                                                                   
     print(f"Predicted stock price for the next day is: {predictions[0][0]}")                                      
                                                                                                                   


                                                   MODEL OUTPUT                                                    

y=x[2] ans=1.0 s1=10 while s1=0: ans+=y if ans1+(s1y) or s1==10.5: ret=10 #print(ans, ret) if -1s1>=10 and         
ret%10==0 and ani==1: continue ans=ain+1 s1+=y elif -1s1>=0 and s1=10: ret=ain+1 ans=ain+1 s1+=y elif -2s1 >=10 and
ret%10==0 and ani1: continue ans=ain+1 s1+=y elif -1s1=10 and ani==1: ret=ain+1 ans=ain+1 s1+=y elif -1s1=10 and   
ret%10==0 and ani1==1: continue ans=ain+1 ans=ain+1 s1+=y a=y+2**s-1 #print(ans, ret)                              
ans=a[:-1]+a[1:-1],ans+s1+a[3:-1],ans+s2+a[2:-1] q=3*s-1[0]+s1q(y+1, x); if q==1 or q%10 == 0: a[0], ans=x+a,      
y+a[0].

## Eng-Spa

In [18]:
dataset = create_instruct_dataset(num_samples, ["eng_spa"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Generating responses... : 100%|██████████| 1/1 [00:01<00:00,  1.65s/it]


INPUT PROMPT                                                    

No he decidido a qué trabajo voy a presentarme. How do you say this sentence in English?                           


                                                   GROUND_TRUTH                                                    

I haven't decided which job to apply for.                                                                          


                                                   MODEL OUTPUT                                                    

I have not decided what to submit.

## PAWS

In [19]:
dataset = create_instruct_dataset(num_samples, ["paws"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Map:   0%|          | 0/49401 [00:00<?, ? examples/s]

Filter:   0%|          | 0/49401 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  2.43it/s]


INPUT PROMPT                                                    

Kabir Suman recorded several albums under the name of Suman Chattopaddhyay or Suman Chatterjee between 1992 and    
1999 . Suman Chatterjee , recorded a number of albums between 1992 and 1999 under the name Suman Chattopaddhyay or 
Kabir Suman . Are these sentences conveying the same meaning? OPTIONS:                                             

 • No                                                                                                              
 • Yes                                                                                                             


                                                   GROUND_TRUTH                                                    

No                                                                                                                 


                                                   MODEL OUTPUT                                                    

No

## Quora

In [14]:
dataset = create_instruct_dataset(num_samples, ["quora"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/56402 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:01<00:00,  1.90s/it]


INPUT PROMPT                                                    

When will liberals realize that the reason Trump is so successful is because he represents everything liberals     
hate?                                                                                                              


                                                   GROUND_TRUTH                                                    

No. Trump just represents HATE, and his worshippers eat it up and spew it out.                                     


                                                   MODEL OUTPUT                                                    

until liberals realize that Trump is the reason Trump is successful

## Alpaca

In [22]:
dataset = create_instruct_dataset(num_samples, ["alpaca"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/52002 [00:00<?, ? examples/s]

Map:   0%|          | 0/20100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:11<00:00, 11.90s/it]


INPUT PROMPT                                                    

Write a function to calculate the factorial of a given number.                                                     


                                                   GROUND_TRUTH                                                    

def factorial(n): if n == 0: return 1 return n * factorial(n-1)                                                    


                                                   MODEL OUTPUT                                                    

a = int(c-1) f = int(a+1) i=1 count = 0 while(i+1=5 and a%i==0) : while i=5: if ai+i!=0: f=f+ii i+=1 count+=1 i+=1 
print(f)